In [1]:
using NamedArrays, DelimitedFiles, DataFrames
using AlgebraOfGraphics, CairoMakie
using SimSpread

[ Info: Precompiling SimSpread [afa5cc60-bf5a-45b9-aa6c-27158a106b2a]
  ** incremental compilation may be fatally broken for this module **



In [2]:
# Load dataset
y = NamedArray(readdlm("../yamanishi2008/DT/nr_DT.txt"))
setnames!(y,["D$i" for i in 1:size(y,1)],1)
setnames!(y,["T$i" for i in 1:size(y,2)],2)

X = NamedArray(readdlm("../yamanishi2008/DD/nr_DD.maccs.txt"))
setnames!(X,["D$i" for i in 1:size(X,1)],1)
setnames!(X,["D$i" for i in 1:size(X,2)],2)

y,X

([0.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [1.0 0.7567567567567568 … 0.45 0.38461538461538464; 0.7567567567567568 1.0 … 0.4 0.3076923076923077; … ; 0.45 0.4 … 1.0 0.35185185185185186; 0.38461538461538464 0.3076923076923077 … 0.35185185185185186 1.0])

In [3]:
# Load dataset
y = read_namedmatrix("../simspread-antifungals/data/DT/antifungals.DT.csv", '\t')
#setnames!(y,["D$i" for i in 1:size(y,1)],1)
#setnames!(y,["T$i" for i in 1:size(y,2)],2)

X = read_namedmatrix("../simspread-antifungals/data/DD/antifungals.ecfp4_tanimoto.txt", ' ')
#setnames!(X,["D$i" for i in 1:size(X,1)],1)
#setnames!(X,["D$i" for i in 1:size(X,2)],2)

y,X

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [1.0 0.6111111111111112 … 0.056818181818181816 0.05555555555555555; 0.6111111111111112 1.0 … 0.07407407407407407 0.07692307692307693; … ; 0.056818181818181816 0.07407407407407407 … 1.0 0.07142857142857142; 0.05555555555555555 0.07692307692307693 … 0.07142857142857142 1.0])

In [4]:
# Split dataset
test, train... = split(y, 10)
train = vcat(train...)

ytest = y[test,:]
Xtest = X[test,train]
ytrain = y[train,:]
Xtrain = X[train,train];

In [5]:
ytest, ytrain

([0.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [6]:
Xtrain, Xtest

([1.0 0.08088235294117647 … 0.09045226130653267 0.06756756756756757; 0.08088235294117647 1.0 … 0.06569343065693431 0.06097560975609756; … ; 0.09045226130653267 0.06569343065693431 … 1.0 0.04666666666666667; 0.06756756756756757 0.06097560975609756 … 0.04666666666666667 1.0], [0.0859375 0.13333333333333333 … 0.06153846153846154 0.06756756756756757; 0.029197080291970802 0.1111111111111111 … 0.04477611940298507 0.02531645569620253; … ; 0.08791208791208792 0.049586776859504134 … 0.13872832369942195 0.078125; 0.14189189189189189 0.15294117647058825 … 0.08387096774193549 0.06862745098039216])

In [7]:
# Featurize description
Xtrain′= featurize(Xtrain, 0.0, false)
Xtest′ = featurize(Xtest, 0.0, false)

9×86 Named Matrix{Float64}
                                                                                               A ╲ B │   …  
─────────────────────────────────────────────────────────────────────────────────────────────────────┼──────
Imipramine                                                                                           │   …  
Robenidine                                                                                           │      
Bleomycin                                                                                            │      
Bepridil                                                                                             │      
Aripiprazole                                                                                         │      
Auranofin                                                                                            │      
Zotepine                                                                                             

In [8]:
# Construct trilayered graph
G = construct(ytrain, ytest, Xtrain′, Xtest′)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [9]:
# Predict interactions
yhat = predict(G, ytest)

9×14 Named Matrix{Float64}
                                                                                               A ╲ B │   …  
─────────────────────────────────────────────────────────────────────────────────────────────────────┼──────
Imipramine                                                                                           │   …  
Robenidine                                                                                           │      
Bleomycin                                                                                            │      
Bepridil                                                                                             │      
Aripiprazole                                                                                         │      
Auranofin                                                                                            │      
Zotepine                                                                                             

In [10]:
trg_degree = k(y')
setnames!(trg_degree, names(y,2), 1)
trg_degree

14×1 Named Matrix{Int64}
                      A ╲ B │  1
────────────────────────────┼───
Aspergillus flavus          │  3
Aspergillus fumigatus       │  8
Aspergillus niger           │  3
Candida albicans            │ 78
Candida glabrata            │ 17
Candida parapsilosis        │ 16
Candida tropicalis          │ 13
Cryptococcus deuterogattii  │  4
Cryptococcus gattii         │  5
Cryptococcus neoformans     │ 22
Fusarium spp                │  7
Penicillium digitatum       │  1
Trichophyton�spp            │  5
Zygomycetes                 │  1

In [11]:
y′ = y[:, [t for t in names(y,2) if t != "Candida albicans"]]
nonempty = first.(Tuple.(findall(!iszero, k(y′))))
y′ = y′[nonempty, :]

43×13 Named Matrix{Float64}
                                                                                 A ╲ B │   …  
───────────────────────────────────────────────────────────────────────────────────────┼──────
(E)-3-(4-Tert-butylphenylsulfonyl)acrylonitrile_[bay_11-7085]                          │   …  
(E)-3-Tosylacrylonitrile_[bay_11-7082]                                                 │      
3-(N-n-Octadecylcarbamoyloxy)-2-methoxypropyl-2-thiazolinium_ethyl_phosphate_[CV-3988] │      
5-Fluorocytosine                                                                       │      
Alexidine_dihydrochloride                                                              │      
Amiodarone                                                                             │      
Amphotericin_B                                                                         │      
Anidulafungin                                                                          │      
Bepridil              

In [12]:
size(y)

(95, 14)

In [13]:
# Evaluate predictions
for (i, rpred) in enumerate(eachrow(yhat))
    println(hcat(rpred, ytest[i,:]))
    println("Ligand '$(test[i])'")
    println("AuPRC = $(SimSpread.AuPRC(Bool.(ytest[i,:]), rpred))")

    println()
end

[0.03321123321123321 0.0; 0.07635241102983038 0.0; 0.03321123321123321 0.0; 0.7961190471660913 1.0; 0.17684894944771698 0.0; 0.16574909424673837 0.0; 0.1339658780670015 0.0; 0.04301075268817196 0.0; 0.05424670774435192 0.0; 0.22447446848935418 0.0; 0.07855238631100703 0.0; 0.011494252873563218 0.0; 0.05594144114097627 0.0; 0.010989010989011009 0.0]
Ligand 'Imipramine'
AuPRC = 0.0

[0.03321123321123321 0.0; 0.07635241102983038 1.0; 0.03321123321123321 0.0; 0.7961190471660913 1.0; 0.17684894944771698 0.0; 0.16574909424673837 0.0; 0.1339658780670015 0.0; 0.04301075268817196 0.0; 0.05424670774435192 0.0; 0.22447446848935418 1.0; 0.07855238631100703 0.0; 0.011494252873563218 0.0; 0.05594144114097627 0.0; 0.010989010989011009 0.0]
Ligand 'Robenidine'
AuPRC = 0.46031746031746035

[0.03321123321123321 0.0; 0.07635241102983038 0.0; 0.03321123321123321 0.0; 0.7961190471660913 1.0; 0.17684894944771698 0.0; 0.16574909424673837 0.0; 0.1339658780670015 0.0; 0.04301075268817196 0.0; 0.054246707744351